### FlshAttention


### 1. GPU内存架构
- 物理内存架构：片上内存与片下内存
  - 片上内存：位于GPU芯片内部，主要作用是高速缓存，共享内容和寄存器
    -  特点：速度极快，存储空间小
    -  硬件类型：通常是SRAM，优点是访问速度快，无需刷新即可保存
 -  片下内存：位于GPU芯片外部，主要用作全局内存，即显存
    -  特点：容量大，速度相对较慢
    -  硬件：通常是HBM，通过讲多个DDR芯片堆叠并于GPU封装，实现大容量和高位宽
-  逻辑内存层次与功能
   -  寄存器：
      -  位置与速度：片上，GPU速度最快的内存空间
      -  作用域：每个线程独享的私有资源，用于存储线程内频繁使用的临时变量
      -  生命周期：与核函数一行周期一致
      -  特点：容量有限
   -  本地内存：
      -  位置与速度：片下，GPU速度最慢的内存空间
      -  作用域：每个线程块内共享的资源，用于存储线程块内频繁使用的临时变量
      -  用途：主要存储编译器无法确定索引的本地数据和因体积过大或数量过多而无法放入寄存器的变量
   -  共享内存：
      -  位置与速度：片上，可编程的内存，访问速度快
      -  作用域：被一个线程块内的所有线程共享，可用于块内线程间的高效通信
      -  生命周期：与线程块一致
   -  全局内存：
      -  位置与速度：片下，可编程的、可扩展的内存，访问速度相对较慢
      -  作用域：所有线程可以访问，用于存储全局数据
      -  生命周期：与程序一致
      -  缓存：对全局内存的访问经过L1和L2缓存提速
   -  常量内存：
      -  位置与速度：片下，不可编程的、可扩展的、只读内存，访问速度相对较慢
      -  作用域：所有线程可以访问，用于存储常量数据
      -  生命周期：与程序一致
      -  用途：主要用于存储在核函数执行期间不会改变的数据
   -  L1/L2缓存：
      -  位置：L1缓存位于每个SM内部，被CUDA核心共享。L2缓存被GPU上所有的SM共享
      -  硬件类型：都是片上SRAM
      -  作用：由系统自动控制, 对程序员不完全透明

### 2.FlashAttention
- 分块： 模型不再一次性处理整句话，而是将巨大的QKV矩阵切成一个个可以塞进SRAM极速缓存的小块，在SRAM内部完成小块的矩阵乘法后，直接进行后续操作
- 在线Softmax
  - 标准softmax需要知道整行数据最大值和总和才能计算归一化因子，迫使模型必须读完一整行数据
  - FlashAttention使用增量式计算，通过维护局部最大值和缩放因子，可以边读数据便更新Softmax结果，不需要等待整行数据读完
- 重计算：在反向传播中，模型通常需要保存前向传播时生成的N*N注意力矩阵，十分耗显存。Flash Attention选择以计算换空间。不存储巨大矩阵，而是反向传播需要时，利用SRAM的分块数据重新算

### 3. Triton算子
- pytorch 原生代码：缺乏算子融合，会导致大量数据在显存和计算单元反复跑路
- Cuda:性能极高，但开发难度巨大
- Triton: 提高一个python接口，允许直接操作线程块，讲更底层的内存合并、共享内存管理和线程同步交给编译器自动处理

#### 1）Triton的核心思想：并行化与分块
并行化：不会写一个成勋来处理整个矩阵，而是写一个矩阵模板，GPU会启动其他程序的实例
- Triton 内核函数一定要加@triton.jit
- 每个内核函数在被实际调用时都是众多实例的一个，确保计算在正确的地址上
- 有了指针地址才能从GPU上加载到数据
- 一些细节比如mask是考虑到了数据块的大小和真实输入的大小  

分块：Triton会自动将输入数据进行分块，然后进行计算，最后将结果进行合并

#### 2) 内部流程
1. 计算偏移量（Offsets）： 你需要利用块大小（BLOCK_SIZE）和当前程序 ID（pid）来计算当前算子负责处理的数据在内存中的起始位置。
2. 边界掩码（Masking）： 由于数据总量可能无法被块大小整除，你需要创建一个布尔掩码（Mask），确保算子不会访问到非法内存区域。
3. 加载数据（Load）： 利用计算好的偏移量和掩码，将数据从慢速的全局显存（HBM）一次性搬运到快速的寄存器中。Triton 会自动处理内存合并（Burst Mode），让这次搬运尽可能快。
4. 计算与写回（Compute & Store）： 在寄存器中完成加减乘除（如 GLU 的非线性变换或 Softmax 的约减），最后将结果写回全局显存